In [86]:
import json
import numpy as np
import pandas as pd
import psycopg2

In [87]:
file_path = 'C:\\Users\\merte\\Desktop\\Mert\CASE\\corrupted-file.json'

In [91]:
try:
    # Read the entire content of the file
    raw_string = "["
    with open(file_path, 'r') as file:        
        for line in file: 
            raw_string += line
        raw_string += "]"
    # process into json
    as_json = json.loads(raw_string)
    rows = []  # içinde ne olduğunu bilmediğim için böyle tanımladım.
    for arr in as_json:
        # get measurements and to through its array # baslıklar dosyadan alındı measurement altından
        for values in arr['measurements']:
            timestamp = values['ts'] # string olarak aldım diğerleri değişebilir
            measurement_name = None
            time_series = None
            measurement = None
    #iç içe iki adet json olduğu için measument altındaki ilk for döngüsünde motorun altındaki başlıkları 2. for döngüsünde inceledik.
            for key in values['series'].keys():
                if key == '$_time':
                    time_series = values['series']['$_time']
                else:
                    measurement = values['series'][key]
                    measurement_name = key
    
            row = pd.DataFrame({
                "title":measurement_name,
                "timestamp":timestamp,
                "time_series":time_series,
                "measurements":measurement
            })
            rows.append(row)

    #merge them all together
    df = pd.concat(rows)

except Exception as e:
    print(f"An error occurred: {e}")

In [89]:
df.to_json("flattened_measurements.json", orient='records')
df.to_csv("flattened_data.csv", index=False)  # CSV olarak kaydet

In [90]:
print("Columns in the flattened DataFrame:")
print(df.head())

Columns in the flattened DataFrame:
                              title                 timestamp  time_series  \
0  FrontPress3_Motor_Vibration.ab-1  2021-02-02T00:59:49.371Z            0   
1  FrontPress3_Motor_Vibration.ab-1  2021-02-02T00:59:49.371Z          101   
2  FrontPress3_Motor_Vibration.ab-1  2021-02-02T00:59:49.371Z          201   
3  FrontPress3_Motor_Vibration.ab-1  2021-02-02T00:59:49.371Z          301   
4  FrontPress3_Motor_Vibration.ab-1  2021-02-02T00:59:49.371Z          402   

   measurements  
0      0.434028  
1      0.438549  
2      0.434028  
3      0.434028  
4      0.432219  


In [83]:
parsed_json = json.loads(df.to_json(orient='records'))

In [85]:
connection = psycopg2.connect (
    host = "localhost",
    database = "data",
    user = "postgres",
    password ="409898"
    )

print("Connection successful")


cur = connection.cursor()
cur.execute("CREATE TABLE data(title TEXT, timestamp TEXT, time_series TEXT,measurements TEXT)")

#data_tuple = (df.to_json[0]('title'), df.to_json[0]('timestamp'), df.to_json[0]('time_series'), df.to_json[0]('measurements'))

for record in parsed_json:
    data_tuple = (record['title'], record['timestamp'], json.dumps(record['time_series']), json.dumps(record['measurements'])) # jsonda olan biçim farkını string e çevirmek için json.dumps kullanılır.
    cur.execute("INSERT INTO data(title, timestamp, time_series, measurements) VALUES (%s, %s, %s, %s)", data_tuple)

#cur.execute("INSERT INTO data(title, timestamp, time_series, measurements) VALUES (%s, %s, %s, %s)", data_tuple)
#cur.execute("INSERT INTO data(title, timestamp, time_series, measurements) VALUES (%s,%s,%s,%s)", (df.to_json(orient='records'), ),(df.to_json(orient='records'), ),(df.to_json(orient='records'), ),(df.to_json(orient='records'), ))

connection.commit()


connection.close()


Connection successful
